In [2]:
#CREACION DE DATA
import random
import string

CANTIDAD = 10

# Función para generar una contraseña aleatoria
def generate_password(length=8):
    characters = string.ascii_letters + string.digits
    return ''.join(random.choice(characters) for _ in range(length))

# Genera CANTIDAD usuarios con contraseñas
users = []
for i in range(CANTIDAD):
    username = f"user{i + 1}"
    password = generate_password()
    users.append(f"{username},{password}")

# Guarda los usuarios en un archivo de texto
with open("D:/MAGUILAR/CURSO-REDIS/TRABAJO/cargamasiva/data.txt", "w") as file:
    file.write("\n".join(users))

print("Usuarios y contraseñas generados y guardados en users.txt.")

Usuarios y contraseñas generados y guardados en users.txt.


In [6]:
#INSTALACION DE REDIS
!pip install redis

   -------------------------------------- 250.3/250.3 kB 902.4 kB/s eta 0:00:00


In [3]:
#CARGA MASIVA DE DATOS
import random
import string
import time
import redis


r = redis.Redis(
    host='redis-16482.c15.us-east-1-2.ec2.cloud.redislabs.com',
    port=16482,
    password='u7kaMTO2MmgRj3gkRe8pvbVSfcn5C2M2'
)

def generate_hash(length=16):
    characters = string.ascii_letters + string.digits
    return ''.join(random.choice(characters) for _ in range(length))

# Abre el archivo de datos
with open('D:/MAGUILAR/CURSO-REDIS/TRABAJO/cargamasiva/data.txt', 'r') as file:
    # Lee línea por línea y carga los datos en Redis
    for line in file:
        username, password = line.strip().split(',')

        if r.hget("users", username):
            print("Lo siento, el usuario {} seleccionado ya está en uso.".format(username))

        #Se obtiene el siguiente userid
        userid = r.incr("next_user_id")
        authsecret = generate_hash()

        #Se añade el usuario a la key users
        r.hset("users", username, userid)

        datos = {
            "username": username,
            "password": password,
            "auth": authsecret
        }

        #Se crea el registro del user:<CORRELATIVO>
        r.hmset("user:{}".format(userid),datos)

        #Se crea un nuevo registro en la key auths
        r.hset("auths", authsecret, userid)

        #Se crea un registro en la key users_by_time
        r.zadd("users_by_time", {username: int(time.time())})

# Cierra la conexión a Redis
r.connection_pool.disconnect()

print("Datos cargados exitosamente en Redis.")


<ipython-input-3-9301da3abd27>:40: DeprecationWarning: Redis.hmset() is deprecated. Use Redis.hset() instead.
  r.hmset("user:{}".format(userid),datos)


Datos cargados exitosamente en Redis.
